# Time resolved NMR data visualization 

This is an extra notebook where you can see animations for the time resolved data of the NMR spectrum. Exciting! To do this we are using a different backend as so (see imports below): 

```python
%matplotlib notebook
```

You do not need to do any programming or writing in this notebook. Just run all cells to see the output. To not confuse the backend, which is a bit finicky - just run the visualization cells (with pyplot/ipywidget code) one at the time. 

In [ ]:
%matplotlib notebook

In [ ]:
from IPython.display import Markdown, HTML
import ipywidgets as widgets 

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation

import logging
logging.getLogger().setLevel(logging.CRITICAL)

## Read in data as in main notebook


In [ ]:
time_r_spectra = pd.read_csv("data/NMR/mixture.csv", header=11, dtype=np.float64) 
time_r_spectra.index = np.array([round(i,3) for i in np.linspace(0, 11, len(time_r_spectra.index))])
time_r_spectra.index.name = r'$\delta$ (ppm)'
time_r_spectra.columns.name = 'Time (s)'
time_r_spectra.index = time_r_spectra.index.astype(np.float64) 
time_r_spectra.columns = time_r_spectra.columns.astype(np.float64) 
time = time_r_spectra.columns.values
ppm = time_r_spectra.index.values

## Fixed time (ipywidgets)
Below we have made an interactive plot using [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html). You can use `ipywidgest` to make interactive plots as below with buttons or with scrollers and a bunch of cool different stuff. When you click on the different buttons you will see the NMR spectra change according to the time it was taken (0 s, 5.051 s or 10 s). 

In [ ]:
def plot(time, ax):
    ax.clear()
    measurement = time_r_spectra[time].values
    ax.plot(ppm, measurement, c="b")
    ax.set_ylabel("Intensity", fontsize=12)
    ax.set_xlabel(r'$\delta$ (ppm)', fontsize=12)
    ax.set_xlim([8.3, 6.8])
    ax.set_ylim([-0.4*10**8, 4.0*10**8])
    plt.show()

toggle = widgets.ToggleButtons(options=['0.000 s', '5.051 s', '10.000 s'], description='Time:', button_style='')
fig, ax = plt.subplots()
def on_click(change):
    plot(float(change['new'][:-2]), ax)
toggle.observe(on_click, 'value')
toggle

### Continues time (matplotlib animation)
Below we have utilized [matplotlib.animation](https://matplotlib.org/3.1.1/api/animation_api.html) to visualize how the NMR spectra changes as a function of time. <br>
Ignore the warning "*No handles with labels found to put in legend.*" and the plot created below the video.

In [ ]:
fig = plt.figure()
ax = plt.axes(xlim=(8.3, 6.8), ylim=(-0.4*10**8, 3.6*10**8))
line, = ax.plot([], [], lw=3)

def init():
    line.set_data([], [])
    return line,

legend = plt.legend()
def animate(i):
    x = ppm
    y = time_r_spectra.iloc[:,i].values
    line.set_data(x, y)
    line.set_label("Time: " + str(time[i]) + " s")
    legend = plt.legend(loc='upper left')
    return line,

anim = FuncAnimation(fig, animate, init_func=init, frames=100, interval=100, blit=True, )
# HTML(anim.to_html5_video())

### 3D plot (matplotlib 3D plotting)

Below we have utilized the 3D plotting capabilities of `matplotlib` to visualize the NMR time resolved data in 3 dimensions. You can rotate it around with the mouse!

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': '3d'}, figsize=(8.0, 6.0)) # figsize=(8.0*2, 6.0*2)

sel = (ppm < 8.4) & (ppm > 6.8)
for t in reversed(time):
    measurement = time_r_spectra[t].values[sel]
    tt = np.full(len(measurement), t)
    ax.plot(tt, ppm[sel], measurement, "b", lw=0.3)
    ax.set_zlabel(r"Intensity", fontsize=17, color= "black")
    ax.set_ylabel(r"$\delta$ (ppm)", fontsize=17, color= "black")
    ax.set_xlabel(r"$Time (s) $", fontsize=17, color= "black")

plt.show()